# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1, shuffle=True, random_state = 42, stratify = data.toxic.to_numpy())

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from razdel import tokenize

def razdel_tokenizer(text):
    return [token.text for token in tokenize(text)]

count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()
count_vectorizer_razdel = CountVectorizer(tokenizer=razdel_tokenizer)
tfidf_vectorizer_razdel = TfidfVectorizer(tokenizer=razdel_tokenizer)
clf = LogisticRegression()

In [5]:
def vectorizing(vectorizer, train, test):

    X = vectorizer.fit_transform(train.comment).toarray()
    X_test = vectorizer.transform(test.comment).toarray()
    y = train.toxic.values
    y_test = test.toxic.values

    return X, X_test, y, y_test

In [6]:
def training(X, X_test, y, y_test, classifier, proba=False):

    clf = classifier
    clf.fit(X, y)
    preds = clf.predict(X_test)
    print(classification_report(y_test, preds, zero_division=0))
    if proba:
        pred_proba = clf.predict_proba(X_test)
        return clf, preds, pred_proba
    return clf, preds

In [7]:
X_count, X_test_count, y_count, y_test_count = vectorizing(count_vectorizer, train, test)
X_count_razdel, X_test_count_razdel, y_count_razdel, y_test_count_razdel = vectorizing(count_vectorizer_razdel, train, test)

C:\Users\StepanovaAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
X_tfidf, X_test_tfidf, y_tfidf, y_test_tfidf = vectorizing(tfidf_vectorizer, train, test)
X_tfidf_razdel, X_test_tfidf_razdel, y_tfidf_razdel, y_test_tfidf_razdel = vectorizing(tfidf_vectorizer_razdel, train, test)

## CountVectorizer

In [9]:
training(X_count, X_test_count, y_count, y_test_count, clf)

              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90       959
         1.0       0.86      0.70      0.77       483

    accuracy                           0.86      1442
   macro avg       0.86      0.82      0.84      1442
weighted avg       0.86      0.86      0.86      1442



(LogisticRegression(), array([1., 0., 0., ..., 0., 0., 0.]))

### razdel

In [10]:
training(X_count_razdel, X_test_count_razdel, y_count_razdel, y_test_count_razdel, clf)

C:\Users\StepanovaAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90       959
         1.0       0.86      0.72      0.78       483

    accuracy                           0.87      1442
   macro avg       0.86      0.83      0.84      1442
weighted avg       0.87      0.87      0.86      1442



(LogisticRegression(), array([1., 0., 0., ..., 0., 0., 0.]))

## TF IDF

In [11]:
training(X_tfidf, X_test_tfidf, y_tfidf, y_test_tfidf, clf)

              precision    recall  f1-score   support

         0.0       0.83      0.96      0.89       959
         1.0       0.88      0.60      0.71       483

    accuracy                           0.84      1442
   macro avg       0.85      0.78      0.80      1442
weighted avg       0.85      0.84      0.83      1442



(LogisticRegression(), array([1., 0., 0., ..., 0., 0., 0.]))

### razdel

In [12]:
training(X_tfidf_razdel, X_test_tfidf_razdel, y_tfidf_razdel, y_test_tfidf_razdel, clf)

              precision    recall  f1-score   support

         0.0       0.83      0.96      0.89       959
         1.0       0.88      0.60      0.72       483

    accuracy                           0.84      1442
   macro avg       0.85      0.78      0.80      1442
weighted avg       0.85      0.84      0.83      1442



(LogisticRegression(), array([1., 0., 0., ..., 0., 0., 0.]))

результат здесь повышается при токенизации не сильно, но кажется, это неудачный сид, раньше результаты значительно улучшались

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\StepanovaAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

def lemmatize_text(text):
  tokens = tokenize(text)
  lemmas = []
  for token in tokens:
    parsed = morph.parse(token.text)[0]
    if "PNCT" not in parsed.tag:
      lemmas.append(parsed.normal_form)
  return " ".join(lemmas)

In [175]:
count_vectorizer = CountVectorizer(stop_words=russian_stopwords, preprocessor=lemmatize_text, ngram_range=(1, 2), min_df=0.0001, max_df=0.99)
X_count, X_test_count, y_count, y_test_count = vectorizing(count_vectorizer, train, test)

C:\Users\StepanovaAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn(


In [176]:
logreg_clf = LogisticRegression(class_weight="balanced", max_iter=110, random_state=42)
logreg_clf, preds, proba = training(X_count, X_test_count, y_count, y_test_count, clf, proba=True)

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.91       959
         1.0       0.86      0.73      0.79       483

    accuracy                           0.87      1442
   macro avg       0.87      0.83      0.85      1442
weighted avg       0.87      0.87      0.87      1442



In [21]:
tfidf_vectorizer = TfidfVectorizer(stop_words=russian_stopwords, preprocessor=lemmatize_text, ngram_range=(1, 2), min_df=0.0001, max_df=0.99)
X_tfidf, X_test_tfidf, y_tfidf, y_test_tfidf = vectorizing(tfidf_vectorizer, train, test)

C:\Users\StepanovaAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn(


In [211]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=50, random_state=42, class_weight="balanced")
forest_clf, preds_forest, proba_forest = training(X_count, X_test_count, y_count, y_test_count, forest_clf, proba=True)

              precision    recall  f1-score   support

         0.0       0.86      0.91      0.88       959
         1.0       0.80      0.70      0.75       483

    accuracy                           0.84      1442
   macro avg       0.83      0.80      0.81      1442
weighted avg       0.84      0.84      0.84      1442



Самые токсичные

In [212]:
test["proba_log"] = proba[:,1]
test["proba_forest"] = proba_forest[:,1]
test.sort_values(by=["proba_log"], ascending=False)

,comment,toxic,proba_log,proba_tree,proba_forest
3016,блеаадить как же обидно когда создавать тред п...,1.0,1.000000e+00,0.0,0.620000
6554,по мексиканск флаг ублюдок мать твой а ну идти...,1.0,1.000000e+00,1.0,0.760000
14224,01 30 о встреча с чубайс 02 40 как шария защищ...,1.0,1.000000e+00,1.0,0.220000
13982,что какой денежный петух нет у хохлов свой ска...,1.0,1.000000e+00,1.0,0.853235
551,перепутать в роддом негритёнок хохол и русский...,1.0,9.999981e-01,1.0,0.900000
...,...,...,...,...,...
12849,есть производитель и ритейл и у каждый свой ос...,0.0,2.149573e-13,1.0,0.060000
13693,росстат сообщить о рекордный за почти два год ...,0.0,6.935072e-16,0.0,0.100000
8813,потому что индустрия игра не благотворительнос...,0.0,1.244532e-19,0.0,0.100000
13973,населённый пункт сша вашингтон город в округ к...,0.0,5.584313e-24,0.0,0.100000


In [213]:
test.sort_values(by=["proba_forest"], ascending=False)

,comment,toxic,proba_log,proba_tree,proba_forest
13589,у они от веганство совсем мозг высохнуть,1.0,0.821210,1.0,1.0
13792,у он хохол быть в предок,1.0,0.897903,1.0,1.0
12343,не русский армянский национальность а россиянка,1.0,0.809592,1.0,1.0
391,vladtime сука что у ты в голова,1.0,0.941156,1.0,1.0
5982,кто угодный но не этот два отброс свинья и нед...,1.0,0.960799,1.0,1.0
...,...,...,...,...,...
9548,за всё коммунальный услуга кроме оприборенный ...,0.0,0.000061,0.0,0.0
8480,я про целесообразность покупка новый лад веста...,0.0,0.000159,0.0,0.0
9111,и всё таки я когда-нибудь посидеть и срисовать...,0.0,0.072918,0.0,0.0
8520,вы описать как надо эксплуатировать ssd для до...,0.0,0.079451,0.0,0.0


- везде в топ-5 вошли токсичные комменты
- однако они не совпадают у разных классификаторов
- но веростность того, что что-то из топа-5 по логрегу принадлежит токсичному классу, согласно лесу, низка, логрег же считает комменты из топа-5 леса довольно токсичными
- более того, комменты, отмеченные токсичными логрегом, кажутся мне более токсичными

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

## logreg

In [200]:
import numpy as np

In [202]:
feature_importances_lr = np.abs(logreg_clf.coef_[0])
feature_names = count_vectorizer.get_feature_names_out()

top_5_lr = sorted(zip(feature_importances_lr, feature_names), reverse=True)[:5]
top_5_lr

[(np.float64(3.3001844755282184), 'хохол'),
 (np.float64(3.128510956782556), 'хохлов'),
 (np.float64(3.01980484354331), 'дебил'),
 (np.float64(2.61924283319946), 'шлюха'),
 (np.float64(2.3412840089571794), 'быдло')]

## tree

In [214]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(criterion='log_loss', random_state=42, class_weight="balanced")
tree_clf, preds_tree, proba_tree = training(X_tfidf, X_test_tfidf, y_tfidf, y_test_tfidf, tree_clf, proba=True)

              precision    recall  f1-score   support

         0.0       0.84      0.80      0.82       959
         1.0       0.63      0.69      0.66       483

    accuracy                           0.76      1442
   macro avg       0.73      0.74      0.74      1442
weighted avg       0.77      0.76      0.76      1442



In [215]:
feature_importances_dt = tree_clf.feature_importances_

top_5_dt = sorted(zip(feature_importances_dt, feature_names), reverse=True)[:5]
top_5_dt 

[(np.float64(0.024551252126889196), 'это'),
 (np.float64(0.01912567738796729), 'хохол'),
 (np.float64(0.016420766775276558), 'год'),
 (np.float64(0.014971973011969552), 'всё'),
 (np.float64(0.0136703148926069), 'свой')]

## RandomForestClassifier

In [35]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=50, random_state=42, class_weight="balanced")
forest_clf, preds_forest, proba_forest = training(X_tfidf, X_test_tfidf, y_tfidf, y_test_tfidf, forest_clf, proba=True)

              precision    recall  f1-score   support

         0.0       0.85      0.90      0.87       959
         1.0       0.78      0.69      0.73       483

    accuracy                           0.83      1442
   macro avg       0.81      0.79      0.80      1442
weighted avg       0.83      0.83      0.83      1442



In [36]:
feature_importances_rf = forest_clf.feature_importances_
top_5_rf = sorted(zip(feature_importances_rf, feature_names), reverse=True)[:5]

In [37]:
top_5_rf

[(np.float64(0.009603642415217472), 'мигрантов'),
 (np.float64(0.008292873661427988), 'библиотек'),
 (np.float64(0.006704594624919413), 'мусорного'),
 (np.float64(0.0065636302393929915), 'атморы'),
 (np.float64(0.0064755951304561465), 'жидовок')]